In [1]:
import requests
import sys
import pandas as pd
import numpy as np
from scipy import sparse
from io import StringIO
import networkscience as ns
from matplotlib import pyplot as plt
import seaborn as sns
reply = requests.get('https://data.europa.eu/euodp/data/storage/f/2020-08-11T140550/Finalised%20mobilities%20started%20in%202018%20-%20KA1.csv')
reply.encoding = 'utf-8'
csv_data = reply.text
df = pd.read_csv(StringIO(csv_data), sep=';',low_memory=False, header=0)
for i, name in enumerate(df.columns):
    print(i, name)

0 Project Reference
1 Academic Year
2 Mobility Start Month
3 Mobility End Month
4 Mobility Duration
5 Activity (mob)
6 Field of Education
7 Participant Nationality
8 Education Level
9 Participant Gender
10 Participant Profile
11 Special Needs
12 Fewer Opportunities
13 Participant Age
14 Sending Country Code
15 Sending City
16 Sending Organization
17 Receiving Country Code
18 Receiving City
19 Receiving Organization
20 Participants


In [2]:
mobility_df = df[df[df.columns[10]]=='Learner']
mobility_df = mobility_df[mobility_df['Activity (mob)'].str.contains('Student')]
mobility_df = mobility_df[~mobility_df['Activity (mob)'].str.contains('traineeship')]
# consideriamo solo l'anno accademico 2018/2019 o 2017/2018?
# i numeri cambiano parecchio di anno in anno, è normale?
#mobility_df = mobility_df[mobility_df['Academic Year'].str.contains('19')]
mobility_df = mobility_df.loc[:, df.columns[np.array((16, 19, 20, 5, 14, 6))]]
print((mobility_df['Field of Education'].nunique())) 
mobility_df['Sending Organization'] = mobility_df['Sending Organization'].str.upper()
mobility_df['Receiving Organization'] = mobility_df['Receiving Organization'].str.upper()

141


In [3]:
mobility_df.drop('Activity (mob)',axis=1, inplace=True)
display(mobility_df)

,Sending Organization,Receiving Organization,Participants,Sending Country Code,Field of Education
174,VORARLBERGER LANDESKONSERVATORIUM,LUNDS UNIVERSITET,1,AT,Music and performing arts
195,NATIONAL MINING UNIVERSITY,MONTANUNIVERSITAT LEOBEN,1,UA,Environmental protection technology
196,NATIONAL MINING UNIVERSITY,MONTANUNIVERSITAT LEOBEN,1,UA,Electronics and automation
197,NATIONAL MINING UNIVERSITY,MONTANUNIVERSITAT LEOBEN,1,UA,Mechanics and metal trades
198,MONTANUNIVERSITAT LEOBEN,THE NATIONAL MINERAL RESOURCES UNIVERSITY (THE...,1,AT,Mining and extraction
...,...,...,...,...,...
667190,UNIVERSITY COLLEGE LONDON,FEDERAL STATE AUTONOMOUS EDUCATIONAL INSTITUTI...,1,UK,Political sciences and civics
667191,UNIVERSITY COLLEGE LONDON,FEDERAL STATE AUTONOMOUS EDUCATIONAL INSTITUTI...,1,UK,Political sciences and civics
667192,FEDERAL STATE AUTONOMOUS EDUCATIONAL INSTITUTI...,UNIVERSITY COLLEGE LONDON,1,RU,Economics
667193,UNIVERSITY COLLEGE LONDON,THE HEBREW UNIVERSITY OF JERUSALEM,1,UK,"Arts and humanities, inter-disciplinary progra..."


In [4]:
mobility_df['Field of Education'] = mobility_df['Field of Education'].str.replace(', not further defined','')
mobility_df['Field of Education'] = mobility_df['Field of Education'].str.replace(', inter-disciplinary programmes','')
mobility_df['Field of Education'] = mobility_df['Field of Education'].str.replace(', not elsewhere classified','')
mobility_df['Field of Education'] = mobility_df['Field of Education'].str.replace(' not elsewhere classified','')
mobility_df['Field of Education'] = mobility_df['Field of Education'].str.replace(', not elsewhere classifed','')
mobility_df['Field of Education'] = mobility_df['Field of Education'].str.replace('Inter-disciplinary programmes and qualifications involving ','')
mobility_df['Field of Education'] = mobility_df['Field of Education'].str.lower()
mobility_df['Field of Education'] = mobility_df['Field of Education'].str.strip()

In [5]:
print(mobility_df['Field of Education'].nunique())
print(np.sort(mobility_df['Field of Education'].unique()))

106
['accounting and taxation' 'agriculture'
 'agriculture, forestry, fisheries and veterinary'
 'architecture and construction' 'architecture and town planning' 'arts'
 'arts and humanities' 'audio-visual techniques and media production'
 'biochemistry' 'biological and related sciences' 'biology'
 'building and civil engineering' 'business and administration'
 'business, administration and law'
 'care of the elderly and of disabled adults'
 'chemical engineering and processes' 'chemistry'
 'child care and youth services' 'community sanitation' 'computer use'
 'crop and livestock production'
 'database and network design and administration' 'dental studies'
 'domestic services' 'earth sciences' 'economics' 'education'
 'education science' 'electricity and energy' 'electronics and automation'
 'engineering and engineering trades'
 'engineering, manufacturing and construction' 'environment'
 'environmental protection technology' 'environmental sciences'
 'fashion, interior and industrial

In [6]:
nodes = np.array(list(set.union(set(mobility_df.loc[:, mobility_df.columns[0]]), set(mobility_df.loc[:, mobility_df.columns[1]]))))
node_position = {}
for i, name in enumerate(nodes):
    node_position[name] = i
nodes_df = pd.DataFrame({'Nodes': nodes})
mobility_df['SendId'] = (np.array(list(map(lambda x: node_position[x], mobility_df['Sending Organization']))))
mobility_df['RecId']= (np.array(list(map(lambda x: node_position[x], mobility_df['Receiving Organization']))))
# Adjacency matrix
mobility_serie = mobility_df.groupby(['SendId', 'RecId']).sum()
row = np.array(mobility_serie.index.get_level_values(1).tolist())
col = np.array(mobility_serie.index.get_level_values(0).tolist())
val = mobility_serie.values
adj_matrix_crs = sparse.csr_matrix((val.flatten(), (row, col)), shape=(nodes.size, nodes.size))
print(ns.fraction_in_giant(adj_matrix_crs))

0.9936995471549518


In [7]:
dd = {}
for field in mobility_df['Field of Education'].unique():
    temp_df = mobility_df.loc[mobility_df['Field of Education'] == field]
    temp_serie = temp_df.groupby(['SendId', 'RecId']).sum()
    row = np.array(temp_serie.index.get_level_values(1).tolist())
    col = np.array(temp_serie.index.get_level_values(0).tolist())
    val = temp_serie.values
    temp_adj = sparse.csr_matrix((val.flatten(), (row, col)), shape=(nodes.size, nodes.size))
    dd[field] = ns.fraction_in_giant(temp_adj)
field_df = pd.DataFrame.from_dict(dd, orient='index', columns=['percentage'])
print('Top', 10, 'for', 'percentage')
pd.set_option('display.max_rows', 10)
field_df = field_df.sort_values('percentage', ascending=False)
display(field_df.head(10))

Top 10 for percentage


,percentage
business and administration,0.403032
engineering and engineering trades,0.290805
management and administration,0.285095
economics,0.281355
information and communication technologies (icts),0.252018
languages,0.235479
political sciences and civics,0.208309
literature and linguistics,0.183698
law,0.179563
arts,0.177200


TODO: Add an iterative version of it, checking if 2 or more field of education can get a percentage of nodes in the giant component close to the one with all the edges (99.36%).
You can also work in the other way around iteratively deleting edges of one field 

In [9]:
import itertools
fields = mobility_df['Field of Education'].unique()
subsets = []
for L in range(1,5):
    for subset in itertools.combinations(fields, L):
        subsets +=[subset,]
print(len(subsets))

5166281
